In [30]:
import pyspark 
from pyspark.sql import SparkSession
from pyspark.sql import types
import pandas as pd 

In [2]:

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/01 17:45:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
!wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-01.csv

In [ ]:
df = \
(
spark.read
    .option('header', 'true')
    .csv('fhvhv_tripdata_2021-01.csv')
)

In [ ]:
df.head(2)

In [ ]:
df.schema

In [ ]:
!head -n 101 fhvhv_tripdata_2021-01.csv > head.csv

In [ ]:
!wc -l head.csv

In [ ]:
df_pandas = pd.read_csv('head.csv')

In [ ]:
df_pandas.dtypes

In [ ]:
spark.createDataFrame(df_pandas).schema

In [ ]:
schema = types.StructType(
[
types.StructField('hvfhs_license_num', types.StringType(),True),
types.StructField('dispatching_base_num', types.StringType(),True),
types.StructField('pickup_datetime', types.TimestampType(),True),
types.StructField('dropoff_datetime', types.TimestampType(),True),
types.StructField('PULocationID', types.IntegerType(),True),
types.StructField('DOLocationID', types.IntegerType(),True),
types.StructField('SR_Flag', types.StringType(),True)
]
)

In [ ]:
df = \
(
spark.read
    .option('header', 'true')
    .schema(schema)
    .csv('fhvhv_tripdata_2021-01.csv')
)

In [ ]:
df.head(10)

In [ ]:
df = df.repartition(24)

In [ ]:
df.write.parquet('fhvhv/2021/01/')

In [3]:
df = spark.read.parquet('fhvhv/2021/01/')

In [5]:
df.head(3)

[Row(hvfhs_license_num='HV0005', dispatching_base_num='B02510', pickup_datetime=datetime.datetime(2021, 1, 4, 8, 24, 29), dropoff_datetime=datetime.datetime(2021, 1, 4, 8, 51, 40), PULocationID=67, DOLocationID=249, SR_Flag=None),
 Row(hvfhs_license_num='HV0005', dispatching_base_num='B02510', pickup_datetime=datetime.datetime(2021, 1, 1, 18, 40, 45), dropoff_datetime=datetime.datetime(2021, 1, 1, 18, 46, 24), PULocationID=161, DOLocationID=162, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02884', pickup_datetime=datetime.datetime(2021, 1, 1, 2, 19, 41), dropoff_datetime=datetime.datetime(2021, 1, 1, 2, 35, 19), PULocationID=18, DOLocationID=167, SR_Flag=None)]

In [6]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0005|              B02510|2021-01-04 08:24:29|2021-01-04 08:51:40|          67|         249|   null|
|           HV0005|              B02510|2021-01-01 18:40:45|2021-01-01 18:46:24|         161|         162|   null|
|           HV0003|              B02884|2021-01-01 02:19:41|2021-01-01 02:35:19|          18|         167|   null|
|           HV0003|              B02871|2021-01-02 12:44:13|2021-01-02 12:50:02|         216|         216|   null|
|           HV0003|              B02869|2021-01-01 09:21:19|2021-01-01 09:36:24|          90|          66|   null|
|           HV0003|              B02866|2021-01-03 15:26:26|2021-01-03 15:48:35|

In [7]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



In [19]:
df.head(10)

[Row(hvfhs_license_num='HV0005', dispatching_base_num='B02510', pickup_datetime=datetime.datetime(2021, 1, 4, 8, 24, 29), dropoff_datetime=datetime.datetime(2021, 1, 4, 8, 51, 40), PULocationID=67, DOLocationID=249, SR_Flag=None),
 Row(hvfhs_license_num='HV0005', dispatching_base_num='B02510', pickup_datetime=datetime.datetime(2021, 1, 1, 18, 40, 45), dropoff_datetime=datetime.datetime(2021, 1, 1, 18, 46, 24), PULocationID=161, DOLocationID=162, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02884', pickup_datetime=datetime.datetime(2021, 1, 1, 2, 19, 41), dropoff_datetime=datetime.datetime(2021, 1, 1, 2, 35, 19), PULocationID=18, DOLocationID=167, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02871', pickup_datetime=datetime.datetime(2021, 1, 2, 12, 44, 13), dropoff_datetime=datetime.datetime(2021, 1, 2, 12, 50, 2), PULocationID=216, DOLocationID=216, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02869', pickup_

In [27]:
print(pyspark.__version__)

3.2.1


In [22]:
(
df
    .select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID')
    .filter(df.hvfhs_license_num == "HV0003")
    .show()
)

+-------------------+-------------------+------------+------------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+-------------------+-------------------+------------+------------+
|2021-01-01 02:19:41|2021-01-01 02:35:19|          18|         167|
|2021-01-02 12:44:13|2021-01-02 12:50:02|         216|         216|
|2021-01-01 09:21:19|2021-01-01 09:36:24|          90|          66|
|2021-01-03 15:26:26|2021-01-03 15:48:35|          89|         225|
|2021-01-05 10:35:42|2021-01-05 10:52:52|          39|         210|
|2021-01-03 16:11:00|2021-01-03 16:16:09|         162|         237|
|2021-01-01 06:02:40|2021-01-01 06:11:43|         260|         129|
|2021-01-01 16:09:29|2021-01-01 16:22:04|         243|          41|
|2021-01-01 05:02:43|2021-01-01 05:08:19|         119|          42|
|2021-01-03 18:13:11|2021-01-03 18:23:40|          79|         140|
|2021-01-04 16:38:50|2021-01-04 17:05:48|          92|         185|
|2021-01-01 15:43:50|2021-01-01 16:42:35|       

# Homework 

In [28]:
!wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv

--2022-03-01 20:11:43--  https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv
Распознаётся nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)… 52.217.107.92
Подключение к nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|52.217.107.92|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 733822658 (700M) [text/csv]
Сохранение в: «fhvhv_tripdata_2021-02.csv»

fhvhv_tripdata_2021 100%[===================>] 699,83M  30,0MB/s    за 2m 19s  

2022-03-01 20:14:02 (5,05 MB/s) - «fhvhv_tripdata_2021-02.csv» сохранён [733822658/733822658]



In [31]:
schema = types.StructType(
[
types.StructField('hvfhs_license_num', types.StringType(),True),
types.StructField('dispatching_base_num', types.StringType(),True),
types.StructField('pickup_datetime', types.TimestampType(),True),
types.StructField('dropoff_datetime', types.TimestampType(),True),
types.StructField('PULocationID', types.IntegerType(),True),
types.StructField('DOLocationID', types.IntegerType(),True),
types.StructField('SR_Flag', types.StringType(),True)
]
)

In [32]:
df = \
(
spark.read
    .option('header', 'true')
    .schema(schema)
    .csv('fhvhv_tripdata_2021-02.csv')
)

In [39]:
df.repartition(24).write.parquet('fhvhv/2021/02/', mode='overwrite')

22/03/01 20:41:39 WARN MemoryManager: Total allocation exceeds 95,00% (1 020 054 720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers
22/03/01 20:41:40 WARN MemoryManager: Total allocation exceeds 95,00% (1 020 054 720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers
22/03/01 20:41:42 WARN MemoryManager: Total allocation exceeds 95,00% (1 020 054 720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers


In [114]:
fhv_tripdata = \
(
spark
    .read
    .parquet('fhvhv/2021/02/')
)

In [75]:
(
df
    .select('pickup_datetime')
    .withColumn('pickup_datetime', df.pickup_datetime.cast('Date'))
    .filter('pickup_datetime == "2021-02-15"')
    .count()
)

367170

In [90]:
(
df.registerTempTable('taxi_data')
)

/usr/local/Cellar/apache-spark/3.2.1/libexec/python/pyspark/sql/dataframe.py:138: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn(


In [91]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02889|2021-02-03 19:14:15|2021-02-03 19:31:36|         198|          63|   null|
|           HV0003|              B02878|2021-02-03 20:03:07|2021-02-03 20:15:09|          47|           3|   null|
|           HV0003|              B02836|2021-02-04 14:41:23|2021-02-04 14:47:41|          65|          97|   null|
|           HV0003|              B02765|2021-02-04 20:07:35|2021-02-04 20:25:03|         255|         113|   null|
|           HV0003|              B02877|2021-02-02 19:19:25|2021-02-02 19:32:24|         142|          90|   null|
|           HV0005|              B02510|2021-02-02 18:32:18|2021-02-02 18:51:48|

In [104]:
spark.sql("""
select 
dispatching_base_num, count(1) as num_records
from taxi_data
group by dispatching_base_num
order by num_records desc 
limit 1
""").collect()

[Row(dispatching_base_num='B02510', num_records=3233664)]

In [120]:
schema_zones = types.StructType(
[
types.StructField('LocationID', types.IntegerType(),True),
types.StructField('Borough', types.StringType(),True),
types.StructField('Zone', types.StringType(),True),
types.StructField('service_zone', types.StringType(),True),
]
)

In [121]:
zones = (
spark
    .read
    .option('header', 'true')
    .schema(schema)
    .csv('taxi+_zone_lookup.csv')
)

In [116]:
fhv_tripdata.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02889|2021-02-03 19:14:15|2021-02-03 19:31:36|         198|          63|   null|
|           HV0003|              B02878|2021-02-03 20:03:07|2021-02-03 20:15:09|          47|           3|   null|
|           HV0003|              B02836|2021-02-04 14:41:23|2021-02-04 14:47:41|          65|          97|   null|
|           HV0003|              B02765|2021-02-04 20:07:35|2021-02-04 20:25:03|         255|         113|   null|
|           HV0003|              B02877|2021-02-02 19:19:25|2021-02-02 19:32:24|         142|          90|   null|
|           HV0005|              B02510|2021-02-02 18:32:18|2021-02-02 18:51:48|

In [124]:
zones.dtypes

[('LocationID', 'int'),
 ('Borough', 'string'),
 ('Zone', 'string'),
 ('service_zone', 'string')]

In [122]:
zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [128]:
(
zones.registerTempTable('zones')
)

In [130]:
spark.sql("""
with popular_pair as (
    select 
    PULocationID, 
    DOLocationID, 
    count(1) as num_records
    from taxi_data
    group by PULocationID, DOLocationID
    order by num_records desc 
    limit 1
)
select 
*
from zones 
where LocationID = (select PULocationID from popular_pair)
""").show()

+----------+--------+-------------+------------+
|LocationID| Borough|         Zone|service_zone|
+----------+--------+-------------+------------+
|        76|Brooklyn|East New York|   Boro Zone|
+----------+--------+-------------+------------+

